# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium
import random

# Import API key
from api_keys import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df["Max Temp"]= city_data_df["Max Temp"]-273.15

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tolanaro,-25.0319,46.9987,22.04,73,20,9.26,MG,1718735367
1,1,kalabo,-14.9700,22.6814,17.95,34,0,1.95,ZM,1718735367
2,2,atafu village,-8.5421,-172.5159,29.26,72,98,6.26,TK,1718735368
3,3,praia da vitoria,38.7333,-27.0667,21.14,64,40,4.22,PT,1718735257
4,4,richards bay,-28.7830,32.0377,19.63,74,0,2.53,ZA,1718735369


---

### Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [20]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    size="Humidity", 
    color="City",
    alpha= 0.5,
    tiles= "OSM",
    scale= 1
)
# Display the map
map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]


# Drop any rows with null values

narrow_city_df = narrow_city_df.dropna()

# Display sample data
narrow_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
55,55,vlore,40.4667,19.4897,25.91,65,0,3.61,AL,1718735387
83,83,pachino,36.7186,15.0907,23.68,71,0,1.31,IT,1718735411
133,133,al bayda,32.7627,21.7551,21.11,59,0,1.59,LY,1718735438
197,197,kuvandyk,51.4781,57.3552,21.15,76,0,2.06,RU,1718735471
293,293,mafinga,-7.2500,35.0667,21.12,49,0,1.23,TZ,1718735519
377,377,mersa matruh,31.3525,27.2453,25.23,77,0,3.33,EG,1718735563
412,412,neka,36.6508,53.2990,25.73,73,0,2.06,IR,1718735581
461,461,prado,-17.3411,-39.2208,25.34,68,0,3.16,BR,1718735605
485,485,ilhabela,-23.7781,-45.3581,25.99,57,0,2.37,BR,1718735573
553,553,kencong,-8.2833,113.3667,22.06,92,0,0.96,ID,1718735649


### Step 3: Create a new DataFrame called hotel_df.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
55,vlore,AL,40.4667,19.4897,65,
83,pachino,IT,36.7186,15.0907,71,
133,al bayda,LY,32.7627,21.7551,59,
197,kuvandyk,RU,51.4781,57.3552,76,
293,mafinga,TZ,-7.2500,35.0667,49,
377,mersa matruh,EG,31.3525,27.2453,77,
412,neka,IR,36.6508,53.2990,73,
461,prado,BR,-17.3411,-39.2208,68,
485,ilhabela,BR,-23.7781,-45.3581,57,
553,kencong,ID,-8.2833,113.3667,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}



# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"]  = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address =  requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vlore - nearest hotel: Hotel Grabova
pachino - nearest hotel: Hotel Celeste
al bayda - nearest hotel: فندق الهديل
kuvandyk - nearest hotel: Отдыхающая локомотивных бригад
mafinga - nearest hotel: No hotel found
mersa matruh - nearest hotel: صقور
neka - nearest hotel: هتل میلاد نکا
prado - nearest hotel: Pousada Recanto do Prado
ilhabela - nearest hotel: Vila Kebaya
kencong - nearest hotel: Grand Gumuk Mas Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
55,vlore,AL,40.4667,19.4897,65,Hotel Grabova
83,pachino,IT,36.7186,15.0907,71,Hotel Celeste
133,al bayda,LY,32.7627,21.7551,59,فندق الهديل
197,kuvandyk,RU,51.4781,57.3552,76,Отдыхающая локомотивных бригад
293,mafinga,TZ,-7.2500,35.0667,49,No hotel found
377,mersa matruh,EG,31.3525,27.2453,77,صقور
412,neka,IR,36.6508,53.2990,73,هتل میلاد نکا
461,prado,BR,-17.3411,-39.2208,68,Pousada Recanto do Prado
485,ilhabela,BR,-23.7781,-45.3581,57,Vila Kebaya
553,kencong,ID,-8.2833,113.3667,92,Grand Gumuk Mas Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    size="Humidity", 
    scale=1,
    color="City",
    alpha= 0.5,
    tiles="OSM",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)